# Hover The Point.

*This notebook will demonstrate more things to include when you hover over some data points*

# 

**Before**: We are used to see the default information ie. X and Y axes on the data point when we hove over it. Of course we can add more data with `text` field in the plot, but still it was limited to that.

**After**: We can tweak the information in our ways to display what and how.

—

**<u>What?</u>**: The thing that goes behind the scenes is, the information that is displayed on the screen is actully represented in `json` and then parsed for spesific information. We can actully get the location of X and Y **cordinates** where the pointer is.

**<u>How?</u>**: Every graph in plotly has one common attribute `hoverData`, which can be accessed and also exported to use it in other ways.

**<u>The Motive Behind Doin All These</u>**: Till here we have seen various DCC / HTML components to work with the inputs from the user and show some relavent information. We have also seen how to get the data and show the image based on the radio inputs. But ***What If*** we want to work / show **information based on the datapoint in the graph itself?**. It think that will give you a nice idea what is going on.

###### 

<center> • • •

<center> Please don't get confused with <code>hovermode</code> and <code>hoverData</code> there are different things. </center>

<center> • • •

Let's get started.

# 

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly.offline as pyo
import plotly.graph_objs as go

import pandas as pd
import json

# 

In [2]:
df = pd.read_csv("../data/data/wheels.csv")
df

,wheels,color,image
0,1,red,redunicycle.jpg
1,1,blue,blueunicycle.jpg
2,1,yellow,yellowunicycle.jpg
3,2,red,redbicycle.jpg
4,2,blue,bluemotorcycle.jpg
5,2,yellow,yellowscooter.jpg
6,3,red,redtricycle.jpg
7,3,blue,bluetricycle.jpg
8,3,yellow,yellowrickshaw.jpg


# Step—By—Step

###### 

### Step 1
*Creting simple structure*

In [5]:
# Creating simple app
app = dash.Dash()

# Generating a layout
app.layout = html.Div([
                html.Div(), # It will hold the graph
                html.Div() # It will show the json
])

# Running server
app.run_server()

Really ↑ this won't do anything. So now, let's do 2nd.

###### 

### Step 2
*Filling in Data*

In [10]:
app = dash.Dash()

# Creating scatter
plot = [go.Scatter(x=df.color,
                  y=df.wheels,
                  mode="markers")]

# Layout which will be passed in figure
layout = go.Layout(title="Hover over and see the result")

# Figure which will be innput in dcc.Graph
figure = go.Figure(data=plot, layout=layout)

app.layout = html.Div([
                html.Div([
                    dcc.Graph(figure=figure,
                              id="graph")
                ]), 
                html.Div([
                    html.Pre(id="preview") # `Pre` is <pre> which will show monospaced
                ])                         # fonts however written in the file.
])

app.run_server()

Cool. Now this shows the scatter plot in a grid like structure and looks like it is working too. Let's do some more stuff.

### Step 3
*Connecting with json and displaying*

In [15]:
app = dash.Dash()

plot = [go.Scatter(x=df.color,
                   y=df.wheels,
                   mode="markers")]
layout = go.Layout(title="Hover over and see the result")
figure = go.Figure(data=plot, layout=layout)

app.layout = html.Div([
                html.Div([
                    dcc.Graph(figure=figure,
                              id="graph")
                ]), 
                html.Div([
                    html.Pre(id="preview")
                ])
])

@app.callback(Output("preview", "children"),
             [Input("graph", "hoverData")])   # Used `hoverData` as the property of graph
                         ### POI ↑ POI ###
def show_values(hovered_data):
    # hovered_data is the json file so need to
    # convert in the simple text
    json_in_text = json.dumps(hovered_data, indent=4)
                ### POI ↑ POI ###
    return json_in_text
    

app.run_server()

**Works!** But if you see, we have two problems. 
1. Not the data of all points work (because hovermode is not closest).
2. The Graph covers all screen (we need to align it to left).

###### 

### Step 4
*Making it smoother*

In [29]:
app = dash.Dash()

plot = [go.Scatter(x=df.color,
                   y=df.wheels,
                   mode="markers")]
layout = go.Layout(title="Hover over and see the result", hovermode="closest")
figure = go.Figure(data=plot, layout=layout)          ### POI ↑ POI ### Solution 1st's

app.layout = html.Div([
                html.Div([
                    dcc.Graph(figure=figure,
                              id="graph")
                ], style={"width":"30%", "float":"left"}), 
                                   ### POI ↑ POI ### Solution 2nd's
                html.Div([
                    html.Pre(id="preview")
                ])
])

@app.callback(Output("preview", "children"),
             [Input("graph", "hoverData")])   
def show_values(hovered_data):
    json_in_text = json.dumps(hovered_data, indent=4)
    return json_in_text

app.run_server()

**Yeah! Now it is ready** <br>
So, for the first time, you've created a graph which can return the json. Cool.

—

**Syntax Analysis**
```python
...,
html.Div([ 
    html.Pre(id="preview")
])
...
```

The `pre` tag will give the monospaced result, because you see, if we were to use `<P>` or `<H1>` or just `<DIV>` they will ignore the style and spaces in json format and will ruin the design. So `<pre>` is useful.

```python
@app.callback(Output(...),
             [Input("graph", "hoverData")])   
                                # ↑ It has json obj
def show_values(hovered_data):
                    # Which ↓ needs to be converted to string
    json_in_text = json.dumps(hovered_data, indent=4)
    return json_in_text          # With some ↑ nice indent
```

###### 

**Extra**
```python
plot = [go.Scatter(..., dy=2)]
                # Somehow ↑ tweaks the spacing on y axis
```

###### 

## Now, let's show image
To show an image, we need to "***access***" the json data.

In [24]:
# We need to import this ↓
import base64

def encode_image(img_path):
    # Simple Byte reading
    with open(img_path, 'rb') as img:
        raw_image = img.read()
        
    # Encoding with base64 and then decode
    encoded = base64.b64encode(raw_image)
    return f'data:image/png;base64,{encoded.decode()}'

In [28]:
app = dash.Dash()

plot = [go.Scatter(x=df.color,
                   y=df.wheels,
                   mode="markers")]
layout = go.Layout(title="Hover over and see the result", hovermode="closest")
figure = go.Figure(data=plot, layout=layout)

app.layout = html.Div([
                html.Div([
                    dcc.Graph(figure=figure,
                              id="graph")
                ], style={"width":"30%", "float":"left"}), 
                                   
                html.Div([
                    html.Img(id="preview") ### POI ← POI ###
                ])
])

@app.callback(Output("preview", "src"), ### POI ← POI ###
             [Input("graph", "hoverData")])   
def show_values(hovered_data):
    if hovered_data:
        data = hovered_data["points"][0]
        wheels = data["y"]
        color = data["x"]
        image_name = df[(df.color == color) & (df.wheels == wheels)].image.values[0]
        img_path = "../data/data/images/" + image_name
        return encode_image(img_path) ### POI ← POI ###

app.run_server()

**Syntax Analysis**
```python
def show_values(hovered_data):
    # We directly can use the dict like syntax
    # on the json obj ↓ as can be seen.
    data = hovered_data["points"][0]
    ...
    return encode_image(img_path)

```

# 

# That's it!
We learned a lot from how and what. Let's go for another ride in the next book.